In [1]:
## EXAMPLE ON PYBULLET

import pybullet as p
import time
import pybullet_data
import numpy as np

"""
physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # optionally
p.setGravity(0, 0, -10)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 1]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
# set the center of mass frame (loadURDF sets base link frame)
for i in range(10000):
    p.stepSimulation()
    time.sleep(1.0 / 240.0)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos, cubeOrn)
p.disconnect() """

## IMPLEMENTATION OF ACROBOT

#   SET THE LENGHT AND HEIGHT OF ACROBOT
lenghFirstLink = 1.0
lenghtSecondLink = 1.0

#   SET THE MASS OF THE FIRST LINK
massFirstLink = 1.0
#   SET THE MASS OF THE SECOND LINK
massSecondLink = 1.0
#   GRAVITY ACCELLERATION
g = 9.81


# DEFINING THE DYNAMIC OF THE SYSTEM INTO A FUNCTION
def dynamic(state, action):
    theta1, theta2, theta1dot, theta2dot = state
    tau = action

    c1 = np.cos(theta1)
    s1 = np.sin(theta1)
    c2 = np.cos(theta2)
    s2 = np.sin(theta2)
    c12 = np.cos(theta1 + theta2)
    s12 = np.sin(theta1 + theta2)

    #   MASS MATRIX
    massMatrix = np.zeros((2, 2))
    # ** is an elevation to 2
    massMatrix[0, 0] = (
        (massFirstLink + massSecondLink) * lenghFirstLink**2
        + massSecondLink * lenghtSecondLink**2
        + 2 * massSecondLink * lenghFirstLink * lenghtSecondLink * c2
    )
    massMatrix[0, 1] = (
        massSecondLink * lenghtSecondLink**2
        + massSecondLink * lenghFirstLink * lenghtSecondLink * c2
    )
    massMatrix[1, 0] = massMatrix[0, 1]
    massMatrix[1, 1] = massSecondLink * lenghtSecondLink**2

    # Coriolis and centrifugal forces
    coriolisForce = np.zeros((2, 2))
    coriolisForce[0, 0] = (
        -2 * massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta2dot
    )
    coriolisForce[0, 1] = (
        -massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta2dot
    )
    coriolisForce[1, 0] = (
        massSecondLink * lenghFirstLink * lenghtSecondLink * s2 * theta1dot
    )
    coriolisForce[1, 1] = 0

    # Gravity forces
    gravityForce = np.zeros((2, 1))
    gravityForce[0, 0] = (
        massFirstLink + massSecondLink
    ) * g * lenghFirstLink * s1 + massSecondLink * g * lenghtSecondLink * s12
    gravityForce[1, 0] = massSecondLink * g * lenghtSecondLink * s12

    # Compute accelerations
    # @ is an operator for matrix multiplication
    tau = np.clip(tau, -10, 10)  # Clip action to prevent large torque values
    accelleration = np.linalg.inv(massMatrix) @ (
        tau - coriolisForce @ np.array([[theta1dot], [theta2dot]]) - gravityForce
    )

    return np.array([theta1dot, theta2dot, accelleration[0, 0], accelleration[1, 0]])

    

# Control policy (swing-up and stabilization)
def control_policy(state, t):
    theta1, theta2, theta1_dot, theta2_dot = state

    # Swing-up control
    if abs(theta1) < np.pi / 2 and abs(theta2) < np.pi / 2:
        return -0.5 * theta1_dot - 1.5 * theta2_dot  # Swing-up torque

    # Stabilization control
    return (
        -10 * theta1 - 5 * theta1_dot - 5 * theta2 - 3 * theta2_dot
    )  
    # Stabilization torque


# Connect to the physics engine
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -g)

# -----------------------------------------------------------------
# Create the Acrobot in the simulation
link1_radius = 0.1
link2_radius = 0.1
link1_height = lenghFirstLink
link2_height = lenghtSecondLink

# Create the floor
planeId = p.loadURDF("plane.urdf")

# Create the base of the Acrobot
base_pos = [0, 0, 0]
base_orient = p.getQuaternionFromEuler([0, 0, 0])
baseId = p.loadURDF("acrobot_total.urdf", base_pos, base_orient)

# -----------------------------------------------------------------

# LOAD ACROBOT
acrobot_id = p.loadURDF("acrobot_total.urdf")

# Enable motors for the joints
p.setJointMotorControl2(acrobot_id, 0, p.POSITION_CONTROL, targetPosition=0, force=0)
p.setJointMotorControl2(acrobot_id, 1, p.VELOCITY_CONTROL, force=0)


# Initial state
theta1_0 = np.random.uniform(low=-np.pi, high=np.pi)
theta2_0 = np.random.uniform(low=-np.pi, high=np.pi)
theta1_dot_0 = np.random.uniform(low=-1.0, high=1.0)
theta2_dot_0 = np.random.uniform(low=-1.0, high=1.0)
state = np.array([theta1_0, theta2_0, theta1_dot_0, theta2_dot_0])

# Simulation parameters
dt = 0.01  # Time step
total_time = 10.0   # Total simulation time

# SIMULATION STEP
for _ in range(int(total_time / dt)):
    # Get the current joint angles and velocities
    joint_states = p.getJointStates(acrobot_id, [0, 1])
    theta1, theta2 = [state[0] for state in joint_states]
    theta1_dot, theta2_dot = [state[1] for state in joint_states]
    
    # Apply the control policy to get the torque
    action = control_policy([theta1, theta2, theta1_dot, theta2_dot], _ * dt)
    
    # Apply the torque to the joints
    p.setJointMotorControl2(acrobot_id, 1, p.TORQUE_CONTROL, force=action)
    
    # Step the simulation
    p.stepSimulation()
    time.sleep(dt)
# GETTING POSITION AND ORIENTATION OF AN ACROBOT
# acroPos, acroOrn = p.getBasePositionAndOrientation(acrobot_id)
# print(acroPos, acroOrn)
# Disconnect from the physics engine
# p.disconnect()